In [ ]:
import fantasy as fp
# noinspection PyUnresolvedReferences
from fantasyalgo import best_subset
import datetime as dt
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd

In [ ]:
# timeperiod of parsing data
start = dt.date(2022, 1, 1)
end = dt.date.today()

# timedelta for getting data, in this case 90 days (3 months)
delta = dt.timedelta(days=90)

# event_id to parse data from
events = [6586, 6140, 6136, 6621, 6370]

dataframes = []
used = []

for key in events:

    event = fp.Event(key)
    # getting players from the event
    players = event.get_players()

    for p in players:
        if p.key not in used:
            used.append(p.key)
            # getting dataset for the player using the parameters above
            dataframes.append(p.get_dataset(start, end, delta))

In [ ]:
# concatenate the frames
data = pd.concat(dataframes)
data = data.dropna()

In [ ]:
data.describe()

In [ ]:
# save dataset
data.to_excel("file_name.xlsx")

In [ ]:
# training data
X = data.drop(columns=['player', 'event', 'pts']).to_numpy()
y = data['pts'].to_numpy()

In [ ]:
event_id = 6970 # your event for predicting
event = fp.Event(event_id)
# getting the players of your event
players = event.get_players()
len(players)

In [ ]:
# save fantasy page with the costs as html file (CTRL + S)
costs_data = event.get_costs("costs_data.html")
len(costs_data)

In [ ]:
# handling different nicknames like hunter and hunter-
for p in players:
    if p.name not in list(costs_data.keys()):
        name = f"{p.name}-"
        if name in list(costs_data.keys()):
            p.name = name
            print(p.name)

In [ ]:
# getting rightly sorted costs list for found players
costs = []
keys = list(costs_data.keys())
not_found = []

cnt = 0
for p in players:
    if p.name in keys:
        costs.append(costs_data[p.name])
    else:
        not_found.append(cnt)
    cnt += 1
len(costs)

In [ ]:
from sklearn.tree import DecisionTreeRegressor

model = DecisionTreeRegressor(max_depth=3)
model.fit(X, y)

In [ ]:
predicted = []
rank = event.rank # get your event rank

for p in players:
    all_rating, all_wr = p.get_stats(dt.date.today() - dt.timedelta(days=90), dt.date.today(), fp.EventFilter.ALL)
    big_rating, big_wr = p.get_stats(dt.date.today() - dt.timedelta(days=90), dt.date.today(), fp.EventFilter.BIG)
    vector = np.array([[rank, all_rating, big_rating, all_wr, big_wr]])
    predicted_linear.append(float(model.predict(vector)[0]))
len(predicted)

In [ ]:
# pass predicted values and costs to get the best 5 players possible
best = best_subset(predicted, costs)

In [ ]:
# print the picked team
for i in best:
    print(f"{players[i].name} --> {best[i]}")

In [ ]:
# explore the results
pred_df['pred pts'] = pred_df['pred pts'].astype(float)
pred_df = pred_df.sort_values(by='pred pts', ascending=False)
pred_df